In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip

import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# !pip install torchvision
import torchvision

import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# !pip install torchmetrics
import torchmetrics
from torchmetrics import Accuracy, Precision, Recall

In [ ]:
batch_size = 60

train_dataset = datasets.MNIST(root="dataset/", download=True, train=True, transform=transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", download=True, train=False, transform=transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for i in range(5):
    image, label = train_dataset[i]
    print(f"Sample {i}: Image shape = {image.shape}, Label = {label}")

In [ ]:
def load_mnist_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        # Read magic number, number of images, rows, and columns
        magic = int.from_bytes(f.read(4), 'big')
        num_images = int.from_bytes(f.read(4), 'big')
        num_rows = int.from_bytes(f.read(4), 'big')
        num_cols = int.from_bytes(f.read(4), 'big')

        # Read the image data
        image_data = np.frombuffer(f.read(), dtype=np.uint8)
        images = image_data.reshape(num_images, num_rows, num_cols)
    return images

train_images = load_mnist_images('dataset/MNIST/raw/train-images-idx3-ubyte.gz')

In [1]:
plt.imshow(train_images[2], cmap='gray')
plt.title("First MNIST Image")
plt.show()

NameError: name 'plt' is not defined

In [ ]:
fig, axes = plt.subplots(5, 10, figsize=(15, 8))
axes = axes.flatten()

for i in range(50):
    axes[i].imshow(train_images[i], cmap='gray')
    axes[i].set_title(f"Image {i}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def imshow(img):
   npimg = img.numpy()
   plt.imshow(np.transpose(npimg, (1, 2, 0)))
   plt.show()

# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)
labels
# show images
imshow(torchvision.utils.make_grid(images))

In [ ]:
class CNN(nn.Module):
   def __init__(self, in_channels, num_classes):

       """
       Building blocks of convolutional neural network.

       Parameters:
           * in_channels: Number of channels in the input image (for grayscale images, 1)
           * num_classes: Number of classes to predict. In our problem, 10 (i.e digits from  0 to 9).
       """
       super(CNN, self).__init__()

       # 1st convolutional layer
       self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, padding=1)
       # Max pooling layer
       self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
       # 2nd convolutional layer
       self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
       # Fully connected layer
       self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

   def forward(self, x):
       """
       Define the forward pass of the neural network.

       Parameters:
           x: Input tensor.

       Returns:
           torch.Tensor
               The output tensor after passing through the network.
       """
       x = F.relu(self.conv1(x))  # Apply first convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = F.relu(self.conv2(x))  # Apply second convolution and ReLU activation
       x = self.pool(x)           # Apply max pooling
       x = x.reshape(x.shape[0], -1)  # Flatten the tensor
       x = self.fc1(x)            # Apply fully connected layer
       return x
       

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = CNN(in_channels=1, num_classes=10).to(device)
print(model)

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs=10
for epoch in range(num_epochs):
 # Iterate over training batches
   print(f"Epoch [{epoch + 1}/{num_epochs}]")

   for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
       data = data.to(device)
       targets = targets.to(device)
       scores = model(data)
       loss = criterion(scores, targets)
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()
print(next(model.parameters()).device, data.device)

In [ ]:
# Print final model device and a test forward pass
print("Model device:", next(model.parameters()).device)
print("Model in eval mode?", not model.training)

# Check one batch
model.eval()
with torch.no_grad():
    test_batch, test_labels = next(iter(test_loader))
    test_batch = test_batch.to(device)
    test_labels = test_labels.to(device)
    out = model(test_batch)
    preds = out.argmax(dim=1)
    print("Test batch shape:", test_batch.shape)
    print("Predictions shape:", preds.shape)
    print("Sample predictions:", preds[:60])
    print("Sample labels:", test_labels[:60])
    print("Sample accuracy on batch:", (preds == test_labels).float().mean().item())

Model device: cuda:0
Model in eval mode? True
Test batch shape: torch.Size([60, 1, 28, 28])
Predictions shape: torch.Size([60])
Sample predictions: tensor([9, 3, 1, 1, 9, 7, 5, 9, 1, 0, 4, 5, 3, 7, 3, 2, 8, 4, 3, 7, 5, 7, 0, 2,
        5, 6, 1, 4, 0, 8, 7, 0, 3, 1, 8, 8, 6, 5, 3, 8, 4, 4, 6, 2, 1, 5, 5, 7,
        6, 1, 9, 1, 7, 5, 8, 9, 5, 7, 3, 1], device='cuda:0')
Sample labels: tensor([9, 3, 1, 1, 9, 7, 5, 4, 1, 0, 4, 5, 3, 7, 3, 2, 8, 4, 3, 7, 5, 7, 0, 2,
        5, 6, 1, 4, 0, 8, 7, 0, 3, 1, 8, 8, 6, 5, 3, 8, 4, 4, 6, 2, 1, 6, 5, 7,
        6, 1, 9, 1, 7, 5, 8, 9, 5, 7, 3, 1], device='cuda:0')
Sample accuracy on batch: 0.9666666984558105


In [ ]:
# Set up of multiclass accuracy, precision, and recall metrics
acc = Accuracy(task="multiclass", num_classes=10).to(device)
precision = Precision(task="multiclass", num_classes=10).to(device)
recall = Recall(task="multiclass", num_classes=10).to(device)

model.eval()
acc.reset(); precision.reset(); recall.reset()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1)
        acc.update(preds, labels)
        precision.update(preds, labels)
        recall.update(preds, labels)

test_accuracy = acc.compute()
test_precision = precision.compute()
test_recall = recall.compute()
print(test_accuracy, test_precision, test_recall)
print(f"Test accuracy: {test_accuracy}")